# Input Data

In [ ]:
#Penginstalan fitur dan library yang akan digunakan
!pip install mip
import numpy as np
import pandas as pd

from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

In [ ]:
#Konversi Data Input ke dalam Matrix

a = pd.read_excel('/content/data penugasan nk.xlsx', sheet_name = 'Sheet1')
a.dropna(inplace=True)
a['Nama'] = a['Nama'].apply(lambda x: x.split(',')[0])
a['MK_no_Kelas'] = a['MK'].apply(lambda x: x.split('_')[0])

def kelas_pemod(row):
  list_dosen = ['']
  if row['MK_no_Kelas'] =='MA3271':
    if row['Nama'] in list_dosen:
      return row['MK_no_Kelas']
    else:
      return np.nan
  else:
    return row['MK_no_Kelas']

a['MK_no_Kelas'] = a.apply(lambda x: kelas_pemod(x),axis=1)
a.dropna(inplace=True)
a.reset_index(inplace=True,drop=True)

crosstab = pd.crosstab(a['Nama'],a['MK'],values=a['SKS'],aggfunc='sum')
crosstab

matrix_jadwal = crosstab.fillna(0).to_numpy()
np.unique(matrix_jadwal,return_counts=True)

crosstab.fillna(0,inplace=True)
crosstab

matrix_jadwal

# Convert DataFrame to Array
data_tes = np.array(crosstab)
# matrix_jadwal
# urutan_kelas = crosstab.columns.tolist()
# urutan_kelas
# urutan_dosen = crosstab.index.tolist()
# urutan_dosen

print(data_tes)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Membuat variabel baru ("data") untuk dipakai di program optimasi (menambahkan baris & kolom index, nama matkul, nama dosen)
size_tes = data_tes.shape
baris_tes = size_tes[0]
kolom_tes = size_tes[1]
matkul = crosstab.columns
nama = crosstab.index

data = np.zeros((baris_tes+2,kolom_tes+2),dtype=object)

for i in range(2,kolom_tes+2):
  data[0,i]=i-1
for i in range(2,baris_tes+2):
  data[i,0]=i-1
for i in range(2,kolom_tes+2):
  data[1,i]=matkul[i-2]
for i in range(2,baris_tes+2):
  data[i,1]=nama[i-2]

data[2:,2:] = data_tes

#Meninjau dimensi data
size = data.shape
baris = size[0]
kolom = size[1]
print(size)
print(data)

df_data = pd.DataFrame(data)
df_data.to_excel(r"/content/drive/MyDrive/Colab Notebooks/df_data_nk.xlsx")

(67, 97)
[[0 0 1 ... 93 94 95]
 [0 0 'AK2263_01' ... 'MA6082_01' 'SK5001_01' 'SK5222_01']
 [1 'L01' 0.0 ... 0.0 0.0 0.0]
 ...
 [63 'L63' 0.0 ... 0.0 0.0 0.0]
 [64 'L64' 0.0 ... 0.0 0.0 0.0]
 [65 'L65' 0.0 ... 0.0 0.0 0.0]]


In [ ]:
#Dictionary Dosen (Ubah codingan karena matriks datanya berubah)
datadosen  = data[2:,1]
indexdosen = data[2:,0]
dosen = {str(indexdosen[i]) : datadosen[i] for i in range(0, len(datadosen))}
#print(dosen)

#Dictionary Matakuliah (Ubah codingan karena matriks datanya berubah)
datamatkul  = data[1,2:]
indexmatkul = data[0,2:]
matkul = {str(indexmatkul[i]) : datamatkul[i] for i in range(0, len(datamatkul))}
#print(matkul)

#Dictionary Hari
harii = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', "Friday"]
hari = {str(i+1) : harii[i] for i in range(0,len(harii))}
#print(hari)

#Dictionary Sesi
sesii = [str(i+7) +'.00 - ' +str(i+8)+'.00' for i in range (11)]
sesi = {str(i+1) : sesii[i]for i in range(0,11)}
#print(sesi)

In [ ]:
#Input Ketersediaan
pd_senin = pd.read_excel('data penugasan nk.xlsx', sheet_name = 'Senin',header = None) 
pd_selasa = pd.read_excel('data penugasan nk.xlsx', sheet_name = 'Selasa', header = None) 
pd_rabu = pd.read_excel('data penugasan nk.xlsx', sheet_name = 'Rabu', header = None) 
pd_kamis = pd.read_excel('data penugasan nk.xlsx', sheet_name = 'Kamis', header = None) 
pd_jumat = pd.read_excel('data penugasan nk.xlsx', sheet_name = 'Jumat', header = None) 

senin = pd_senin.fillna(0).to_numpy()
selasa = pd_selasa.fillna(0).to_numpy()
rabu = pd_rabu.fillna(0).to_numpy()
kamis = pd_kamis.fillna(0).to_numpy()
jumat = pd_jumat.fillna(0).to_numpy()

# Model Optimization

In [ ]:
from mip import Model, xsum, minimize, BINARY

#Pendefinisian indeks
nd = baris-2 ;nm = kolom-2 ;nh = 5 ;ns = 11

#Catatan : Jumlah SKS dosen ekuivalen dengan banyak sesi dosen mengajar

#Pendefinisian koefisien di fungsi objektif
x_k = np.array([10 for i in range (nd*nm*nh*ns)])

#SoftCosntraint 1 : Kelas diuusahakan tidak ada pada pukul 12.00 - 13.00, 16.00 - 17.00, dan 17.00-18.00
for i in range (nd):
  for j in range (nm):
    for k in range (nh):
      x_k[(i)*ns*nh*nm+(j)*ns*nh+(k)*ns+(5)] = 1000
      x_k[(i)*ns*nh*nm+(j)*ns*nh+(k)*ns+(9)] = 1000
      x_k[(i)*ns*nh*nm+(j)*ns*nh+(k)*ns+(10)] = 1000

#Sotfconstraint 2: Kelas diusahakan ada berdasarkan inputan dosen
for i in range(ns):
  for j in range(nd):
    if senin[i+1,j+1] == 0.1:
      for k in range(nm):
        x_k[(j)*ns*nh*nm+(k)*ns*nh+(0)*ns+(i)] = x_k[(j)*ns*nh*nm+(k)*ns*nh+(0)*ns+(i)]*0.1

for i in range(ns):
  for j in range(nd):
    if selasa[i+1,j+1] == 0.1:
      for k in range(nm):
        x_k[(j)*ns*nh*nm+(k)*ns*nh+(1)*ns+(i)] = x_k[(j)*ns*nh*nm+(k)*ns*nh+(1)*ns+(i)]*0.1

for i in range(ns):
  for j in range(nd):
    if rabu[i+1,j+1] == 0.1:
      for k in range(nm):
        x_k[(j)*ns*nh*nm+(k)*ns*nh+(2)*ns+(i)] = x_k[(j)*ns*nh*nm+(k)*ns*nh+(2)*ns+(i)]*0.1

for i in range(ns):
  for j in range(nd):
    if kamis[i+1,j+1] == 0.1:
      for k in range(nm):
        x_k[(j)*ns*nh*nm+(k)*ns*nh+(3)*ns+(i)] = x_k[(j)*ns*nh*nm+(k)*ns*nh+(3)*ns+(i)]*0.1

for i in range(ns):
  for j in range(nd):
    if jumat[i+1,j+1] == 0.1:
      for k in range(nm):
        x_k[(j)*ns*nh*nm+(k)*ns*nh+(4)*ns+(i)] = x_k[(j)*ns*nh*nm+(k)*ns*nh+(4)*ns+(i)]*0.1

##Softconstraint 3: Kelas diusahakan tidak ada berdasarkan inputan dosen
for i in range(ns):
  for j in range(nd):
    if senin[i+1,j+1] == 10:
      for k in range(nm):
        x_k[(j)*ns*nh*nm+(k)*ns*nh+(0)*ns+(i)] = x_k[(j)*ns*nh*nm+(k)*ns*nh+(0)*ns+(i)]*10

for i in range(ns):
  for j in range(nd):
    if selasa[i+1,j+1] == 10:
      for k in range(nm):
        x_k[(j)*ns*nh*nm+(k)*ns*nh+(1)*ns+(i)] = x_k[(j)*ns*nh*nm+(k)*ns*nh+(1)*ns+(i)]*10

for i in range(ns):
  for j in range(nd):
    if rabu[i+1,j+1] == 10:
      for k in range(nm):
        x_k[(j)*ns*nh*nm+(k)*ns*nh+(2)*ns+(i)] = x_k[(j)*ns*nh*nm+(k)*ns*nh+(2)*ns+(i)]*10

for i in range(ns):
  for j in range (nd):
    if kamis[i+1,j+1] == 10:
      for k in range(nm):
        x_k[(j)*ns*nh*nm+(k)*ns*nh+(3)*ns+(i)] = x_k[(j)*ns*nh*nm+(k)*ns*nh+(3)*ns+(i)]*10

for i in range(ns):
  for j in range (nd):
    if jumat[i+1,j+1] == 10:
      for k in range(nm):
        x_k[(j)*ns*nh*nm+(k)*ns*nh+(4)*ns+(i)] = x_k[(j)*ns*nh*nm+(k)*ns*nh+(4)*ns+(i)]*10

#Penggantian konstanta Matkul sks ganjil di sesi 3,6,9 untuk pengurutan
for i in range (nd):
  for j in range (2,kolom):           
    if int(data[i+2,j])%2 != 0:
      m = int(data[0,j])
      for k in range (nh):
        for l in range (2,ns,3):              
          x_k[(i)*ns*nh*nm+(m-1)*ns*nh+(k)*ns+(l)] = x_k[(i)*ns*nh*nm+(m-1)*ns*nh+(k)*ns+(l)]*3  

model = Model("knapsack")

#Pendefinisian variabel
x = [model.add_var(var_type=BINARY) for i in range (nd*nm*nh*ns)]

#Pendefinisian Fungsi Objektif
model.objective = minimize(xsum(x_k[i] * x[i] for i in range (nd*nm*nh*ns)))  

#Hardcosntraint 4 : Setiap dosen mengajar sesuai dengan mata kuliah yang ditugaskan dan jumlah SKS yang diberikan
brobro = 0
for i in range (nd):                  #Looping untuk setiap dosen
  for j in range (2,kolom):           #Looping penentuan mata kuliah yang ditugaskan beserta sks
    if int(data[i+2,j]) > 0:
      m = int(data[0,j])              #Indeks untuk mata kuliah yang ditugaskan
      sks_matkul = int(data[i+2,j])   #Jumlah sks mata kuliah yang ditugaskan
      brobro = brobro + sks_matkul
      model += xsum(xsum(x[(i)*ns*nh*nm+(m-1)*ns*nh+(h)*ns+(s)] for s in range(ns))for h in range (nh)) == sks_matkul

#Hardconstraint 5 : Setiap mata kuliah berlangsung maksimal dua sesi secara berurutan pada suatu hari
for i in range (nd):
  for j in range (nm):
    for k in range (nh):
      model += xsum(x[(i)*ns*nh*nm+(j)*ns*nh+(k)*ns+(s)] for s in range (ns)) <= 2 #Maksimal 2 sesi doang

for i in range (nd):
  for j in range (2,kolom):           
    if int(data[i+2,j]) > 0 and int(data[i+2,j])%2 == 0:
      m = int(data[0,j])
      for k in range (nh):
        for l in range (0,ns-1,2):              
          model += (x[(i)*ns*nh*nm+(m-1)*ns*nh+(k)*ns+(l+1)] - x[(i)*ns*nh*nm+(m-1)*ns*nh+(k)*ns+(l)]) == 0  #Pengurutan mata kuliah SKS genap
          model += x[(i)*ns*nh*nm+(m-1)*ns*nh+(k)*ns+(ns-1)] == 0    #sesi akhir dipaksa 0 karena jumlah sesi ganjil

for i in range (nd):
  for j in range (2,kolom):           
    if int(data[i+2,j])%2 != 0:
      m = int(data[0,j])
      for k in range (nh):
        for l in range (0,ns-1,3):    #iterasi lompat-lompat 3 biar ngasih spare waktu kosong di tengah-tengah, jadinya jadwal lebih enak              
          model += (x[(i)*ns*nh*nm+(m-1)*ns*nh+(k)*ns+(l+1)] - x[(i)*ns*nh*nm+(m-1)*ns*nh+(k)*ns+(l)]) == 0  #Pengurutan mata kuliah SKS ganjil


#Hardconstraint 6 : Setiap Dosen hanya bisa mengajar 1 mata kuliah di setiap sesi setiap harinya.
for i in range (nd):
  for j in range (nh):
    for k in range (ns):
      model += xsum(x[(i)*ns*nh*nm+(m)*ns*nh+(j)*ns+(k)] for m in range (nm)) <= 1


#Hardconstraint 7 : Setiap mata kuliah wajib antar dan per tingkat tidak boleh bentrok (bermasalah, harus <= 1)
for j in range(nh):
  for k in range (ns):
    #model += xsum(xsum(x[(i)*ns*nh*nm+(m)*ns*nh+(j)*ns+(k)] for m in [0,1,2,5,7,8,29,45,56,58])for i in range(nd)) <= 1 #AK [0,1,2,5,7,8,29,45,57,59]
    model += xsum(xsum(x[(i)*ns*nh*nm+(m)*ns*nh+(j)*ns+(k)] for m in [40,42,45,50,54])for i in range(nd)) <= 1 #MA [40,42,45,50,55]


#Hardconstraint 8 : Setiap mata kuliah yang kode tingkatnya sama, tidak boleh ada pada hari dan sesi yang sama.
for j in range(nh):
  for k in range (ns):
    model += xsum(xsum(x[(i)*ns*nh*nm+(m)*ns*nh+(j)*ns+(k)] for m in range (0,3))for i in range(nd)) <= 3       #AK Tingkat 2 1
    model += xsum(xsum(x[(i)*ns*nh*nm+(m)*ns*nh+(j)*ns+(k)] for m in range (3,6))for i in range(nd)) <= 3       #AK Tingkat 3 1
    model += xsum(xsum(x[(i)*ns*nh*nm+(m)*ns*nh+(j)*ns+(k)] for m in range (6,9))for i in range(nd)) <= 3       #AK Tingkat 4 2
    model += xsum(xsum(x[(i)*ns*nh*nm+(m)*ns*nh+(j)*ns+(k)] for m in range (9,11))for i in range(nd)) <= 3      #AK Tingkat 5 1
    model += xsum(xsum(x[(i)*ns*nh*nm+(m)*ns*nh+(j)*ns+(k)] for m in range (11,15))for i in range(nd)) <= 3     #AK Tingkat 6 1
    model += xsum(xsum(x[(i)*ns*nh*nm+(m)*ns*nh+(j)*ns+(k)] for m in [28,29,34,35,38,40,42,44,45,48])for i in range(nd)) <= 3   #MA 2 Tingkat 2 [28,29,34,35,38,40,42,44,45,48]
    model += xsum(xsum(x[(i)*ns*nh*nm+(m)*ns*nh+(j)*ns+(k)] for m in [50,51,52,53,54,56,58])for i in range(nd)) <= 3  #MA 3 Tingkat 3 [50,51,53,54,55,57,59]
    model += xsum(xsum(x[(i)*ns*nh*nm+(m)*ns*nh+(j)*ns+(k)] for m in range (60,67))for i in range(nd)) <= 3   #MA Tingkat 4 2 (61,68)
    model += xsum(xsum(x[(i)*ns*nh*nm+(m)*ns*nh+(j)*ns+(k)] for m in range (67,89))for i in range(nd)) <= 3  #MA Tingkat 5 2 (68,90)
    model += xsum(xsum(x[(i)*ns*nh*nm+(m)*ns*nh+(j)*ns+(k)] for m in range (89,93))for i in range(nd)) <= 3   #MA Tingkat 6 1 (90,94)
    model += xsum(xsum(x[(i)*ns*nh*nm+(m)*ns*nh+(j)*ns+(k)] for m in range (93,95))for i in range(nd)) <= 3   #SK Tingkat 5 2 (94,96)

#Hardconstraint 9 : Kelas Paralel memiliki jadwal yang sama (bermasalah, alin)
for j in range (nm):
  if (j<15 or j>27) :
    for k in range (j+1,nm):
      for l in range (nh):
        for m in range (ns):
          if data[1,j+2][0:6]==data[1,k+2][0:6] :
            model += xsum(x[(i)*ns*nh*nm+(j)*ns*nh+(l)*ns+(m)]for i in range (nd)) - xsum(x[(i)*ns*nh*nm+(k)*ns*nh+(l)*ns+(m)]for i in range (nd)) == 0


#Hardconstraint 10 : Ketersediaan Dosen
for i in range(ns):
  for j in range (nd):
    if senin[i+1,j+1] == 1:
      for k in range(nm):
        model += x[(j)*ns*nh*nm+(k)*ns*nh+(0)*ns+(i)] == 0

for i in range(ns):
  for j in range (nd):
    if selasa[i+1,j+1] == 1:
      for k in range(nm):
        model += x[(j)*ns*nh*nm+(k)*ns*nh+(1)*ns+(i)] == 0

for i in range(ns):
  for j in range (nd):
    if rabu[i+1,j+1] == 1:
      for k in range(nm):
        model += x[(j)*ns*nh*nm+(k)*ns*nh+(2)*ns+(i)] == 0

for i in range(ns):
  for j in range (nd):
    if kamis[i+1,j+1] == 1:
      for k in range(nm):
        model += x[(j)*ns*nh*nm+(k)*ns*nh+(3)*ns+(i)] == 0

for i in range(ns):
  for j in range (nd):
    if jumat[i+1,j+1] == 1:
      for k in range(nm):
        model += x[(j)*ns*nh*nm+(k)*ns*nh+(4)*ns+(i)] == 0


#Hardconstraint 11 : Untuk setiap hari di setiap sesi, tidak boleh lebih dari n mata kuliah
for j in range (nh):
  for k in range (ns):
    model += xsum(xsum(x[(i)*ns*nh*nm+(m)*ns*nh+(j)*ns+(k)] for m in range (nm)) for i in range(nd)) <= 7 


#Hardconstraint 12 : Mata kuliah yang diajar oleh 2 dosen atau lebih, memiliki jadwal yang sama
for i in range (2, kolom):
  d = []
  for j in range (2, baris):
    if data[j,i] != 0:
      d.append(j-2)
  for p in range (len(d)-1):
    for j in range (nh):
      for k in range (ns):
        model += x[d[p+1]*ns*nh*nm+(i-2)*ns*nh+(j)*ns+(k)]- x[d[p]*ns*nh*nm+(i-2)*ns*nh+(j)*ns+(k)] == 0 


#Hardconstraint 13 : Ada mata kuliah yang sudah ditentukan jadwalnya (matkul tingkat 1 dilaksanakan dibawah jam 11)
for j in range(15,28):
    model += xsum(xsum(xsum(x[(i)*ns*nh*nm+(j)*ns*nh+(k)*ns+(l)] for l in range(4,11))for k in range (nh))for i in range (nd)) == 0

#Optimasi menggunakan fungsi dari library MIP
model.optimize()

selected = [i for i in range (nd*nm*nh*ns) if x[i].x >= 0.99]

#Mengeluarkan hasil indeks yang terpilih
print("selected items: {}".format(selected))

selected items: [456, 487, 488, 3146, 3147, 3172, 3173, 6721, 6722, 6756, 13329, 13338, 13339, 18156, 18157, 18200, 18201, 20243, 20244, 20278, 21848, 21849, 21870, 21871, 23711, 23712, 23727, 23728, 27737, 27738, 29878, 29896, 29897, 34161, 34162, 34177, 34178, 40370, 40371, 40383, 44785, 44786, 44796, 44797, 49099, 49100, 49112, 55772, 55773, 55785, 55786, 57821, 57827, 57828, 60119, 60120, 67447, 67448, 67479, 72963, 72964, 72979, 74362, 74363, 74406, 74407, 74716, 74717, 76579, 82285, 82291, 82292, 85377, 85378, 91704, 91713, 91714, 99234, 99235, 99261, 100608, 100609, 100630, 100631, 104211, 104214, 104215, 108737, 108752, 108753, 110552, 110553, 110574, 110575, 114144, 114147, 114148, 116617, 116618, 117288, 117289, 117299, 117300, 122929, 122930, 124275, 124295, 124296, 129272, 129273, 129288, 129710, 129711, 129736, 133566, 133567, 133577, 133578, 134813, 134847, 134848, 138083, 138084, 138094, 138095, 144115, 144116, 144126, 144127, 148022, 148023, 152539, 152540, 152550, 1525

In [ ]:
print(len(selected))
print(brobro)

335
335


# Output

In [ ]:
#Konversi hasil optimasi ke penggunaan indeks awal
var_x = []
for i in range (nd):
  for j in range (nm):
    for k in range (nh):
      for l in range (ns):
        kata = "Dosen " + str(i+1) + " mengajar mata kuliah " +str(j+1)+ " pada hari "+str(k+1)+" di sesi " + str(1+l)
        var_x.append(kata)



for i in range (len(selected)):
  print(var_x[selected[i]])

Dosen 1 mengajar mata kuliah 9 pada hari 2 di sesi 6
Dosen 1 mengajar mata kuliah 9 pada hari 5 di sesi 4
Dosen 1 mengajar mata kuliah 9 pada hari 5 di sesi 5
Dosen 1 mengajar mata kuliah 58 pada hari 2 di sesi 1
Dosen 1 mengajar mata kuliah 58 pada hari 2 di sesi 2
Dosen 1 mengajar mata kuliah 58 pada hari 4 di sesi 5
Dosen 1 mengajar mata kuliah 58 pada hari 4 di sesi 6
Dosen 2 mengajar mata kuliah 28 pada hari 2 di sesi 1
Dosen 2 mengajar mata kuliah 28 pada hari 2 di sesi 2
Dosen 2 mengajar mata kuliah 28 pada hari 5 di sesi 3
Dosen 3 mengajar mata kuliah 53 pada hari 2 di sesi 9
Dosen 3 mengajar mata kuliah 53 pada hari 3 di sesi 7
Dosen 3 mengajar mata kuliah 53 pada hari 3 di sesi 8
Dosen 4 mengajar mata kuliah 46 pada hari 1 di sesi 7
Dosen 4 mengajar mata kuliah 46 pada hari 1 di sesi 8
Dosen 4 mengajar mata kuliah 46 pada hari 5 di sesi 7
Dosen 4 mengajar mata kuliah 46 pada hari 5 di sesi 8
Dosen 4 mengajar mata kuliah 84 pada hari 1 di sesi 4
Dosen 4 mengajar mata kuliah 84

'\nCatatan:\n- Mata kuliah yang sama tapi beda kelas jadwalnya berbeda\n- Mata kuliah yang sama tapi diajar oleh lebih dari 1 dosen jadwalnya berbeda\n'

In [ ]:
#Konversi hasil optimasi ke penggunaan indeks awal (dictionary)
var_x = []
for i in range (nd):
  for j in range (nm):
    for k in range (nh):
      for l in range (ns):
        kata = "Dosen " + dosen[str(i+1)] + " mengajar mata kuliah " +matkul[str(j+1)]+ " pada hari "+hari[str(k+1)]+" di sesi " + sesi[str(1+l)]
        var_x.append(kata)


for i in range (len(selected)):
  print(var_x[selected[i]])

Dosen L01 mengajar mata kuliah AK4285_01 pada hari Tuesday di sesi 12.00 - 13.00
Dosen L01 mengajar mata kuliah AK4285_01 pada hari Friday di sesi 10.00 - 11.00
Dosen L01 mengajar mata kuliah AK4285_01 pada hari Friday di sesi 11.00 - 12.00
Dosen L01 mengajar mata kuliah MA3261_02 pada hari Tuesday di sesi 7.00 - 8.00
Dosen L01 mengajar mata kuliah MA3261_02 pada hari Tuesday di sesi 8.00 - 9.00
Dosen L01 mengajar mata kuliah MA3261_02 pada hari Thursday di sesi 11.00 - 12.00
Dosen L01 mengajar mata kuliah MA3261_02 pada hari Thursday di sesi 12.00 - 13.00
Dosen L02 mengajar mata kuliah MA1203_01 pada hari Tuesday di sesi 7.00 - 8.00
Dosen L02 mengajar mata kuliah MA1203_01 pada hari Tuesday di sesi 8.00 - 9.00
Dosen L02 mengajar mata kuliah MA1203_01 pada hari Friday di sesi 9.00 - 10.00
Dosen L03 mengajar mata kuliah MA3023_01 pada hari Tuesday di sesi 15.00 - 16.00
Dosen L03 mengajar mata kuliah MA3023_01 pada hari Wednesday di sesi 13.00 - 14.00
Dosen L03 mengajar mata kuliah MA302

In [ ]:
#Konversi hasil optimasi ke Output per Dosen

jadwal_dosen = np.zeros((nd),dtype=object)
a = 0 #Inisisasi index variabel selected

for i in range (nd):  
  jadwal = np.zeros((ns,nh),dtype=object)
  for j in range (nm):
    for k in range (nh):
      for l in range (ns):
        if a == len(selected):
          break 
        if selected[a] == (i)*nm*nh*ns+(j)*nh*ns+(k)*ns+(l):
          jadwal[l,k] = matkul[str(j+1)]
          a = a + 1
  jadwal_dosen[i] = jadwal

for i in range (nd):
  df_jadwal_dosen = pd.DataFrame(jadwal_dosen[i], index = sesii , columns = harii) 
  print(dosen[str(i+1)] + " Lecturer Shcedule" + " : " + "\n",df_jadwal_dosen )

L01 Lecturer Shcedule : 
               Monday    Tuesday Wednesday   Thursday     Friday
7.00 - 8.00        0  MA3261_02         0          0          0
8.00 - 9.00        0  MA3261_02         0          0          0
9.00 - 10.00       0          0         0          0          0
10.00 - 11.00      0          0         0          0  AK4285_01
11.00 - 12.00      0          0         0  MA3261_02  AK4285_01
12.00 - 13.00      0  AK4285_01         0  MA3261_02          0
13.00 - 14.00      0          0         0          0          0
14.00 - 15.00      0          0         0          0          0
15.00 - 16.00      0          0         0          0          0
16.00 - 17.00      0          0         0          0          0
17.00 - 18.00      0          0         0          0          0
L02 Lecturer Shcedule : 
               Monday    Tuesday Wednesday Thursday     Friday
7.00 - 8.00        0  MA1203_01         0        0          0
8.00 - 9.00        0  MA1203_01         0        0      

In [ ]:
#Konversi hasil optimasi ke Ouput per hari
a=0
jadwal_hari = np.zeros((nh),dtype=object)

for y in range(nh):
  jadwal = np.zeros((ns),dtype=object)
  jadwal_hari[y] = jadwal

for i in range (nd):  
  for j in range (nm):
    for k in range (nh):
      for l in range (ns):
        if a == len(selected):
          break 
        if selected[a] == (i)*nm*nh*ns+(j)*nh*ns+(k)*ns+(l):
          if jadwal_hari[k][l] == 0:
            jadwal_hari[k][l]=matkul[str(j+1)]
          else:
            jadwal_hari[k][l]=jadwal_hari[k][l] +',' +matkul[str(j+1)]
          a = a + 1
    
for y in range(nh):
  df_jadwal_hari = pd.DataFrame(jadwal_hari[y], index = sesii,columns = ['Mata Kuliah'])
  print("Jadwal hari " + hari[str(y+1)] + " : " + "\n",df_jadwal_hari )

#Export ke google drive
df_senin = pd.DataFrame(jadwal_hari[0], index = sesii,columns = ['Mata Kuliah'])
df_selasa = pd.DataFrame(jadwal_hari[1], index = sesii,columns = ['Mata Kuliah'])
df_rabu = pd.DataFrame(jadwal_hari[2], index = sesii,columns = ['Mata Kuliah'])
df_kamis = pd.DataFrame(jadwal_hari[3], index = sesii,columns = ['Mata Kuliah'])
df_jumat = pd.DataFrame(jadwal_hari[4], index = sesii,columns = ['Mata Kuliah'])
df_senin.to_excel(r"/content/drive/MyDrive/Colab Notebooks/df_senin.xlsx")
df_selasa.to_excel(r"/content/drive/MyDrive/Colab Notebooks/df_selasa.xlsx")
df_rabu.to_excel(r"/content/drive/MyDrive/Colab Notebooks/df_rabu.xlsx")
df_kamis.to_excel(r"/content/drive/MyDrive/Colab Notebooks/df_kamis.xlsx")
df_jumat.to_excel(r"/content/drive/MyDrive/Colab Notebooks/df_jumat.xlsx")


Jadwal hari Monday : 
                                                      Mata Kuliah
7.00 - 8.00    MA5023_01,MA2041_03,MA5272_01,MA6082_01,MA4281...
8.00 - 9.00    MA5023_01,MA2041_03,MA5272_01,MA6082_01,MA4281...
9.00 - 10.00   MA4251_01,MA1201_08,MA5221_03,MA1201_01,MA1201...
10.00 - 11.00  MA5271_01,MA4251_01,MA1201_08,MA1201_01,MA1201...
11.00 - 12.00  MA5271_01,MA2281_01,MA3011_01,MA4072_01,MA4072...
12.00 - 13.00  MA2281_01,MA5031_01,MA3011_01,MA4072_01,MA4072...
13.00 - 14.00  MA2271_01,MA3022_01,MA3022_01,MA2271_02,MA3022...
14.00 - 15.00  MA2271_01,MA3022_01,MA3022_01,MA2271_02,MA3022...
15.00 - 16.00  MA5217_01,MA5041_01,AK2263_01,MA5217_01,MA6052...
16.00 - 17.00                                          MA5273_01
17.00 - 18.00                                          MA5273_01
Jadwal hari Tuesday : 
                                                      Mata Kuliah
7.00 - 8.00    MA3261_02,MA1203_01,MA5031_01,MA3261_01,MA4272...
8.00 - 9.00    MA3261_02,MA1203_01,MA5031_0

In [ ]:
#Konversi hasil optimasi ke Output per Tingkat
jadwal_tingkat = np.zeros((6),dtype=object)
a=0 #Inisisasi index variabel selected
for y in range(6):
  jadwal = np.zeros((ns,nh),dtype=object)
  jadwal_tingkat[y] = jadwal
for i in range (nd):  
  for j in range (nm):
    for k in range (nh):
      for l in range (ns):
        if a == len(selected):
          break 
        if selected[a] == (i)*nm*nh*ns+(j)*nh*ns+(k)*ns+(l):
          for y in range(6):
            if matkul[str(j+1)][2] == str(y+1):
              if jadwal_tingkat[y][l,k] == 0:
                jadwal_tingkat[y][l,k]=matkul[str(j+1)]
              else:
                jadwal_tingkat[y][l,k]=jadwal_tingkat[y][l,k] +',' +matkul[str(j+1)]
          a = a + 1

for y in range(6):
  locals()["df_jadwal_tingkat"+str(y)] = pd.DataFrame(jadwal_tingkat[y], index = sesii , columns = harii)
  print("Jadwal tingkat " + str(y+1) + " : " + "\n",locals()["df_jadwal_tingkat"+str(y)] )

df_jadwal_tingkat0.to_excel(r"/content/drive/MyDrive/Colab Notebooks/df_jadwal_tingkat1.xlsx")
df_jadwal_tingkat1.to_excel(r"/content/drive/MyDrive/Colab Notebooks/df_jadwal_tingkat2.xlsx")
df_jadwal_tingkat2.to_excel(r"/content/drive/MyDrive/Colab Notebooks/df_jadwal_tingkat3.xlsx")
df_jadwal_tingkat3.to_excel(r"/content/drive/MyDrive/Colab Notebooks/df_jadwal_tingkat4.xlsx")
df_jadwal_tingkat4.to_excel(r"/content/drive/MyDrive/Colab Notebooks/df_jadwal_tingkat5.xlsx")
df_jadwal_tingkat5.to_excel(r"/content/drive/MyDrive/Colab Notebooks/df_jadwal_tingkat6.xlsx")

Jadwal tingkat 1 : 
                                                 Monday  ...                                             Friday
7.00 - 8.00                                  MA1202_01  ...                                                  0
8.00 - 9.00                                  MA1202_01  ...                                                  0
9.00 - 10.00   MA1201_08,MA1201_01,MA1201_07,MA1201_11  ...  MA1203_01,MA1201_08,MA1201_07,MA1201_05,MA1201_06
10.00 - 11.00  MA1201_08,MA1201_01,MA1201_07,MA1201_11  ...            MA1201_08,MA1201_07,MA1201_05,MA1201_06
11.00 - 12.00                                        0  ...                                                  0
12.00 - 13.00                                        0  ...                                                  0
13.00 - 14.00                                        0  ...                                                  0
14.00 - 15.00                                        0  ...                                

In [ ]:
yes = []
for i in range(10):
  locals()["x" + str(i)] = i  # use globals() to store the vars in the global stack
  yes.append(locals()["x" + str(i)])
print(yes)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
